In [ ]:
import torch
from torch import optim,nn
from torch.autograd import Variable
import numpy as np
import time
import os
os.chdir('..')
from models.data_loader import DataLoader
from models.retain_time import RETAIN

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
# hyperparameters
batch_size = 50
max_seq_length = 300
min_seq_length = 5
emb_size = 128
hid_size = 128
num_classes = 267+1
cuda_flag = True

In [ ]:
# data loader
D = DataLoader(batch_size=batch_size,
   data_dir='data/batches/',
    mode='val', max_seq_length=max_seq_length, min_seq_length=min_seq_length)

# load model
model = RETAIN(emb_size,hid_size,num_classes,cuda_flag)
if cuda_flag:
    model.cuda()

In [ ]:
for num in np.arange(500,15000,500):
    if cuda_flag:
        ver = 'retain_time_%d.pth'%num
    else:
        ver = 'retain_bi_%d_cpu.pth'%num
    model.load_state_dict(torch.load(os.path.join('data/saved_weights',ver)))
    model.eval()
    # test model
    cnt = 0
    total = 0
    correct = 0
    correct_5 = 0
    
    for i,file in enumerate(D.val_list):
        file_num = int(file.split('_')[1].split('.')[0])
        D.batch_size = int(20000/file_num)
        D.load_batch_file(file)
        for i in range(D.batch_count):
            cnt+=1
            input_list, targets, dates = D.get_batch()
            dates = Variable(torch.Tensor(dates), requires_grad=False)
            if cuda_flag:
                dates = dates.cuda()
            inputs = model.list_to_tensor(input_list)
            outputs = model(inputs,dates)
            targets = torch.LongTensor(targets).view(len(inputs),-1)[:,-1] # to only use last of each sequence
            
#             print(type(targets))
            if cuda_flag:
                targets = targets.cuda()
            total+=len(inputs)
            targets = targets.tolist()
            for tup in zip(targets,outputs.topk(5)[1].data.tolist()):
                if tup[0]==tup[1][0]:
                    correct +=1
                if tup[0] in tup[1]:
                    correct_5 +=1
    print(ver)
    print("Top-1 accuracy: %d/%d = %1.3f"%(correct,total,correct*1.0/total))
    print("Top-5 accuracy: %d/%d = %1.3f"%(correct_5,total,correct_5*1.0/total))